In [1]:
import os 
import glob
from pathlib import Path
import pandas as pd 
import numpy as np 
import nibabel as nib 
import json
from nilearn import plotting, image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [49]:
pd.set_option('display.max_colwidth', 1000)

# 1 Map the segmentations (named for fla image) to their corresponding patient: 

Begin by reading in the json file and data frames that describe what we just segmented: 

In [2]:
PATH = Path('/working/lupolab/julia/tcia_analysis/datasets/')

In [3]:
df = pd.read_csv(Path(PATH, 'one_exam_per_patient_with_fixed_vol.csv'))

In [6]:
with open(Path(PATH.parent,'nvidia', 'config', 'config_datalist_flair.json'), 'r') as f: 
    segment_json = json.load(f)

In [7]:
segment_json

{'training': [],
 'validation': [{'image': 'TCGA-GBM-nifti/TCGA-06-0240/07-02-2005-04728/4-AxFLAIR-thin_for_surgery-47036a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0143/02-11-2005-83875/6-AXIAL_FLAIR-72192a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0149/03-25-2003-87536/5-AXIAL_T2_FLAIR-94682a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-19-0955/04-07-2001-NR_MRI_BRAIN_WWO-83151/3-MRHG_FLAIR_AX-38966a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0648/01-20-2006-51419/6-AXIAL_FLAIR-80030a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-12-0829/06-02-1999-BRAIN-09460/4-FLAIR-82367a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0171/08-04-2001-68007/7-AXIAL_FLAIR_RF2__150-15878a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-14-1401/04-18-1997-MRI_BRAIN_W_WOUT_CONTRAST-27628/601-BRAIN_WWO_FLAIR-20508a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-02-0033/05-26-1997-MRI_BRAIN_WWO_CONTR-89744/4-AX_FLAIR-20679a.nii.gz'},
  {'image': 'TCGA-GBM-nifti/TCGA-06-0127/03-07-2000-83889/4-AXIAL_FLAIR-569

Now list all of the segmentations that are in our 'eval_fla' folder: 

In [8]:
segmentations = [Path(x).stem for x in glob.glob(str(Path(PATH.parent, 'nvidia', 'eval_fla', '*a'))) if x not in glob.glob(str(Path(PATH.parent, 'nvidia', 'eval_fla', '*flaa')))]

In [9]:
len(segmentations)

224

In [10]:
segmentations[:10]

['4-AxFLAIR-thin_for_surgery-47036a',
 '6-AXIAL_FLAIR-72192a',
 '5-AXIAL_T2_FLAIR-94682a',
 '3-MRHG_FLAIR_AX-38966a',
 '6-AXIAL_FLAIR-80030a',
 '4-FLAIR-82367a',
 '7-AXIAL_FLAIR_RF2__150-15878a',
 '601-BRAIN_WWO_FLAIR-20508a',
 '4-AX_FLAIR-20679a',
 '4-AXIAL_FLAIR-56917a']

In [11]:
df.head()

,patient_id,cohort,patient_exam,image,contrast,contrast_label,plane,exam_eligible,fixed_vol
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,9-AxT1-thin_for_surgery-27598,1,T1C,ax,1,4-AxFLAIR-thin_for_surgery-47036
1,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,21-nordicICE_HFH_-_rBF_map_-Leakage_corrected-...,1,rBF,cor,1,4-AxFLAIR-thin_for_surgery-47036
2,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,1-3_PLANE_LOC-22369,0,LOCALIZER,NaN,1,4-AxFLAIR-thin_for_surgery-47036
3,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,5-AXIAL_DIFFUSION-62807,0,DWI,ax,1,4-AxFLAIR-thin_for_surgery-47036
4,TCGA-06-0240,TCGA-GBM,07-02-2005-04728,8-AXIAL_PERFUSION-44568,1,PERFUSION,ax,1,4-AxFLAIR-thin_for_surgery-47036


In [12]:
df.shape

(3630, 9)

In [13]:
df_nodup = df.drop_duplicates(subset=['patient_id'])
df_nodup = df_nodup.drop(['image', 'contrast', 'contrast_label', 'plane', 'exam_eligible', 'fixed_vol'], axis = 1)

In [14]:
df_nodup.shape

(274, 3)

In [15]:
df_nodup = df_nodup.reset_index(drop = True)

In [16]:
df_nodup.head()

,patient_id,cohort,patient_exam
0,TCGA-06-0240,TCGA-GBM,07-02-2005-04728
1,TCGA-06-0143,TCGA-GBM,02-11-2005-83875
2,TCGA-06-0149,TCGA-GBM,03-25-2003-87536
3,TCGA-19-0955,TCGA-GBM,04-07-2001-NR_MRI_BRAIN_WWO-83151
4,TCGA-06-0648,TCGA-GBM,01-20-2006-51419


In [17]:
seg_loc_root = Path(PATH.parent, 'nvidia', 'eval_fla')

In [18]:
seg_list = [x['image'] for x in segment_json['validation']]

In [19]:
seg_list

['TCGA-GBM-nifti/TCGA-06-0240/07-02-2005-04728/4-AxFLAIR-thin_for_surgery-47036a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0143/02-11-2005-83875/6-AXIAL_FLAIR-72192a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0149/03-25-2003-87536/5-AXIAL_T2_FLAIR-94682a.nii.gz',
 'TCGA-GBM-nifti/TCGA-19-0955/04-07-2001-NR_MRI_BRAIN_WWO-83151/3-MRHG_FLAIR_AX-38966a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0648/01-20-2006-51419/6-AXIAL_FLAIR-80030a.nii.gz',
 'TCGA-GBM-nifti/TCGA-12-0829/06-02-1999-BRAIN-09460/4-FLAIR-82367a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0171/08-04-2001-68007/7-AXIAL_FLAIR_RF2__150-15878a.nii.gz',
 'TCGA-GBM-nifti/TCGA-14-1401/04-18-1997-MRI_BRAIN_W_WOUT_CONTRAST-27628/601-BRAIN_WWO_FLAIR-20508a.nii.gz',
 'TCGA-GBM-nifti/TCGA-02-0033/05-26-1997-MRI_BRAIN_WWO_CONTR-89744/4-AX_FLAIR-20679a.nii.gz',
 'TCGA-GBM-nifti/TCGA-06-0127/03-07-2000-83889/4-AXIAL_FLAIR-56917a.nii.gz',
 'TCGA-GBM-nifti/TCGA-19-1390/03-20-2001-NR_MRI_BRAIN_WWO-90326/3-MRHG_FLAIR_AX-64301a.nii.gz',
 'TCGA-GBM-nifti/TCGA-08-0356/03-26-2002-

In [20]:
attempted_segmentations = pd.DataFrame({'patient_id': [x.split('/')[1] for x in seg_list], 
                                        'cohort': [x.split('/')[0] for x in seg_list], 
                                        'patient_exam': [x.split('/')[2] for x in seg_list], 
                                        
             'seg_loc': [x.split('/')[3].split('.')[-3] for x in seg_list], 
             'seg_file': [ x.split('/')[3].split('.')[-3]+'_TC.nii.gz' for x in seg_list]})

In [21]:
actual_segmentations = pd.DataFrame(segmentations)

In [22]:
actual_segmentations.shape, attempted_segmentations.shape

((224, 1), (234, 5))

In [23]:
actual_segmentations.head()

,0
0,4-AxFLAIR-thin_for_surgery-47036a
1,6-AXIAL_FLAIR-72192a
2,5-AXIAL_T2_FLAIR-94682a
3,3-MRHG_FLAIR_AX-38966a
4,6-AXIAL_FLAIR-80030a


In [24]:
attempted_segmentations.head()

,patient_id,cohort,patient_exam,seg_loc,seg_file
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,4-AxFLAIR-thin_for_surgery-47036a,4-AxFLAIR-thin_for_surgery-47036a_TC.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,6-AXIAL_FLAIR-72192a,6-AXIAL_FLAIR-72192a_TC.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,5-AXIAL_T2_FLAIR-94682a,5-AXIAL_T2_FLAIR-94682a_TC.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,3-MRHG_FLAIR_AX-38966a,3-MRHG_FLAIR_AX-38966a_TC.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,6-AXIAL_FLAIR-80030a,6-AXIAL_FLAIR-80030a_TC.nii.gz


In [25]:
actual_seg_map = attempted_segmentations.loc[attempted_segmentations.seg_loc.isin(list(actual_segmentations[0]))]

In [26]:
actual_seg_map.shape

(224, 5)

In [27]:
actual_seg_map['fla'] = [x[:-10]+'.nii.gz' for x in actual_seg_map.seg_file]

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
actual_seg_map.head()

,patient_id,cohort,patient_exam,seg_loc,seg_file,fla
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,4-AxFLAIR-thin_for_surgery-47036a,4-AxFLAIR-thin_for_surgery-47036a_TC.nii.gz,4-AxFLAIR-thin_for_surgery-47036a.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,6-AXIAL_FLAIR-72192a,6-AXIAL_FLAIR-72192a_TC.nii.gz,6-AXIAL_FLAIR-72192a.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,5-AXIAL_T2_FLAIR-94682a,5-AXIAL_T2_FLAIR-94682a_TC.nii.gz,5-AXIAL_T2_FLAIR-94682a.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,3-MRHG_FLAIR_AX-38966a,3-MRHG_FLAIR_AX-38966a_TC.nii.gz,3-MRHG_FLAIR_AX-38966a.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,6-AXIAL_FLAIR-80030a,6-AXIAL_FLAIR-80030a_TC.nii.gz,6-AXIAL_FLAIR-80030a.nii.gz


In [29]:
tcia_df = actual_seg_map

# 2 Shift the segmentations to match the original header 

In [30]:
base_dest_path = '/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/'

In [31]:
PATH_TCIA = Path("/working/lupolab/julia/tcia_analysis")

Find the original paths for the images: 

In [32]:
fla_file_paths = []
for idx, row in tcia_df.iterrows():
    if type(row['fla']) != float: 
        orig_fla_file_path = str(Path(PATH_TCIA, 'datasets',row['cohort'], row['patient_id'], row['patient_exam'],  row['fla']))
        fla_file_paths.append(orig_fla_file_path)

In [33]:
os.chdir(base_dest_path)

In [37]:
# !rm -r *_fla

Read in the original image and the segmentation, shift the segmentation & save it in the same folder as the segmentation: 

In [38]:
from nilearn.image import new_img_like

In [39]:
for idx, row in actual_seg_map.iterrows():
    original_file     = Path(PATH_TCIA, 'datasets', row['cohort'], row['patient_id'], row['patient_exam'], row['fla'])
    segmentation_file = Path(PATH_TCIA, 'nvidia', 'eval_fla', row['seg_loc'], row['seg_file']) 
    orig_fla_nii      = nib.load(original_file)
    seg_fla_nii       = nib.load(segmentation_file)
    shifted_nifti     = new_img_like(orig_fla_nii, 
                                     seg_fla_nii.get_fdata(), 
                                     affine=orig_fla_nii.affine, 
                                     copy_header=True)
    # Ensuring that the shapes match: 
    if shifted_nifti.shape == orig_fla_nii.shape:
        print(f"Shapes match for {row['patient_id']}")
    else: print(f"Shapes don't match for {row['patient_id']}") 
    
    # Save the shifted nifti in the folder where the original segmentation was: 
    shifted_seg_name = Path(segmentation_file.parent, row['seg_file'][:-7]+'_shifted'+row['seg_file'][-7:])
    print(f'Saving {shifted_seg_name}')
    nib.save(shifted_nifti, Path(segmentation_file.parent, shifted_seg_name))

Shapes match for TCGA-06-0240
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AxFLAIR-thin_for_surgery-47036a/4-AxFLAIR-thin_for_surgery-47036a_TC_shifted.nii.gz
Shapes match for TCGA-06-0143
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-72192a/6-AXIAL_FLAIR-72192a_TC_shifted.nii.gz
Shapes match for TCGA-06-0149
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/5-AXIAL_T2_FLAIR-94682a/5-AXIAL_T2_FLAIR-94682a_TC_shifted.nii.gz
Shapes match for TCGA-19-0955
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-MRHG_FLAIR_AX-38966a/3-MRHG_FLAIR_AX-38966a_TC_shifted.nii.gz
Shapes match for TCGA-06-0648
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-80030a/6-AXIAL_FLAIR-80030a_TC_shifted.nii.gz
Shapes match for TCGA-12-0829
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-FLAIR-82367a/4-FLAIR-82367a_TC_shifted.nii.gz
Shapes match for TCGA-06-0171
Saving /working/lupolab/julia/tcia_analysis/nv

Shapes match for TCGA-06-0166
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-AXIAL_FLAIR-96643a/3-AXIAL_FLAIR-96643a_TC_shifted.nii.gz
Shapes match for TCGA-06-0188
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AXIAL_FLAIR-83725a/4-AXIAL_FLAIR-83725a_TC_shifted.nii.gz
Shapes match for TCGA-06-0182
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/7-AxFLAIR-thin_for_surgery-31783a/7-AxFLAIR-thin_for_surgery-31783a_TC_shifted.nii.gz
Shapes match for TCGA-02-0037
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AX_FLAIR-95660a/4-AX_FLAIR-95660a_TC_shifted.nii.gz
Shapes match for TCGA-08-0360
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/7-AX_FLAIR_gad-51944a/7-AX_FLAIR_gad-51944a_TC_shifted.nii.gz
Shapes match for TCGA-19-2624
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-MRHR_FLAIR_AX-17941a/3-MRHR_FLAIR_AX-17941a_TC_shifted.nii.gz
Shapes match for TCGA-06-0154
Saving /working/lupolab/julia/tcia_analysis/

Shapes match for TCGA-06-0241
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/5-AXIAL_FLAIR-99930a/5-AXIAL_FLAIR-99930a_TC_shifted.nii.gz
Shapes match for TCGA-08-0392
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AX_FLAIR-87676a/4-AX_FLAIR-87676a_TC_shifted.nii.gz
Shapes match for TCGA-27-2524
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-T2_FLAIR_COR-11521a/3-T2_FLAIR_COR-11521a_TC_shifted.nii.gz
Shapes match for TCGA-14-0789
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/1101-quick_brainT2FLAIR-02375a/1101-quick_brainT2FLAIR-02375a_TC_shifted.nii.gz
Shapes match for TCGA-14-0783
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/401-Brain______FLAIR_AX_____TRA____TIR___11000_____120___-76548a/401-Brain______FLAIR_AX_____TRA____TIR___11000_____120___-76548a_TC_shifted.nii.gz
Shapes match for TCGA-06-0142
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/7-AXIAL_FLAIR-05489a/7-AXIAL_FLAIR-05489a_TC_shifted.nii

Shapes match for TCGA-DU-8164
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AXIAL_FLAIR-03012a/4-AXIAL_FLAIR-03012a_TC_shifted.nii.gz
Shapes match for TCGA-EZ-7265A
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-FLAIRAXBLADE-09682a/3-FLAIRAXBLADE-09682a_TC_shifted.nii.gz
Shapes match for TCGA-FG-A4MT
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-FLAIR_AXIALS-02285a/3-FLAIR_AXIALS-02285a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7604
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/7-Ax_Flair_irFSE_H-59891a/7-Ax_Flair_irFSE_H-59891a_TC_shifted.nii.gz
Shapes match for TCGA-DU-5853
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-84159a/6-AXIAL_FLAIR-84159a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7684
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/7-Ax_Flair_irFSE_H-26694a/7-Ax_Flair_irFSE_H-26694a_TC_shifted.nii.gz
Shapes match for TCGA-FG-A6J1
Saving /working/lupolab/julia/tcia_analysis/n

Shapes match for TCGA-FG-6689
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-FLAIR_AXIALS-99471a/3-FLAIR_AXIALS-99471a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7856
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-Ax_Flair_irFSE_H-08103a/6-Ax_Flair_irFSE_H-08103a_TC_shifted.nii.gz
Shapes match for TCGA-DU-7019
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-36919a/6-AXIAL_FLAIR-36919a_TC_shifted.nii.gz
Shapes match for TCGA-DU-6408
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-AXIAL_FLAIR-22793a/3-AXIAL_FLAIR-22793a_TC_shifted.nii.gz
Shapes match for TCGA-DU-5874
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/8-AXIAL_FLAIR-99641a/8-AXIAL_FLAIR-99641a_TC_shifted.nii.gz
Shapes match for TCGA-HT-8111
Saving /working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-Ax_Flair_irFSE_H-04644a/6-Ax_Flair_irFSE_H-04644a_TC_shifted.nii.gz
Shapes match for TCGA-HT-7688
Saving /working/lupolab/julia/tcia_analysis/nvid

## 2.2 Check to make sure that all the segmentations have a corresponding shifted segmentation: 

In [40]:
segmentations = glob.glob(base_dest_path+'/*/*a_TC.nii.gz')
len(segmentations)

224

In [41]:
segmentations = glob.glob(base_dest_path+'/*/*a_TC_shifted.nii.gz')
len(segmentations)

224

## 2.3 Change the data frame to point to the shifted segmentations instead of the original ones: 

In [42]:
actual_seg_map['seg_file_shifted'] = [x[:-7]+'_shifted'+x[-7:] for x in actual_seg_map.seg_file]

/working/lupolab/julia/anaconda/envs/pytnew/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
actual_seg_map = actual_seg_map.drop(['seg_file'], axis = 1)


In [50]:
actual_seg_map.head()

,patient_id,cohort,patient_exam,seg_loc,fla,seg_file
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AxFLAIR-thin_for_surgery-47036a,4-AxFLAIR-thin_for_surgery-47036a.nii.gz,4-AxFLAIR-thin_for_surgery-47036a_TC_shifted.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-72192a,6-AXIAL_FLAIR-72192a.nii.gz,6-AXIAL_FLAIR-72192a_TC_shifted.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/5-AXIAL_T2_FLAIR-94682a,5-AXIAL_T2_FLAIR-94682a.nii.gz,5-AXIAL_T2_FLAIR-94682a_TC_shifted.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-MRHG_FLAIR_AX-38966a,3-MRHG_FLAIR_AX-38966a.nii.gz,3-MRHG_FLAIR_AX-38966a_TC_shifted.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-80030a,6-AXIAL_FLAIR-80030a.nii.gz,6-AXIAL_FLAIR-80030a_TC_shifted.nii.gz


In [45]:
actual_seg_map.columns = ['patient_id', 'cohort', 'patient_exam', 'seg_loc', 'fla', 'seg_file']

In [46]:
seg_df = actual_seg_map

In [47]:
seg_df['seg_loc'] = [str(Path(seg_loc_root, x)) for x in list(seg_df['seg_loc'])]

In [51]:
seg_df.head()

,patient_id,cohort,patient_exam,seg_loc,fla,seg_file
0,TCGA-06-0240,TCGA-GBM-nifti,07-02-2005-04728,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/4-AxFLAIR-thin_for_surgery-47036a,4-AxFLAIR-thin_for_surgery-47036a.nii.gz,4-AxFLAIR-thin_for_surgery-47036a_TC_shifted.nii.gz
1,TCGA-06-0143,TCGA-GBM-nifti,02-11-2005-83875,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-72192a,6-AXIAL_FLAIR-72192a.nii.gz,6-AXIAL_FLAIR-72192a_TC_shifted.nii.gz
2,TCGA-06-0149,TCGA-GBM-nifti,03-25-2003-87536,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/5-AXIAL_T2_FLAIR-94682a,5-AXIAL_T2_FLAIR-94682a.nii.gz,5-AXIAL_T2_FLAIR-94682a_TC_shifted.nii.gz
3,TCGA-19-0955,TCGA-GBM-nifti,04-07-2001-NR_MRI_BRAIN_WWO-83151,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/3-MRHG_FLAIR_AX-38966a,3-MRHG_FLAIR_AX-38966a.nii.gz,3-MRHG_FLAIR_AX-38966a_TC_shifted.nii.gz
4,TCGA-06-0648,TCGA-GBM-nifti,01-20-2006-51419,/working/lupolab/julia/tcia_analysis/nvidia/eval_fla/6-AXIAL_FLAIR-80030a,6-AXIAL_FLAIR-80030a.nii.gz,6-AXIAL_FLAIR-80030a_TC_shifted.nii.gz


In [52]:
seg_df.to_csv(Path(PATH, 'segmentation_fla_df.csv'), index = False)

In [53]:
os.listdir(PATH)

['.ipynb_checkpoints',
 'TCGA-GBM-nifti',
 'TCGA-LGG-nifti',
 'come_back_to',
 'manifest_files',
 'dicom_images_with_contrast.csv',
 'contrast_labels.csv',
 'contrast_labels_with_flair_eligibility.csv',
 'contrast_labels_with_eligibility.csv',
 'images_with_dicom_contrast.csv',
 'exam_counts.csv',
 'exam_counts.txt',
 'one_exam_per_patient_with_labels_flair.csv',
 'one_exam_per_patient_with_fixed_vol.csv',
 'one_exam_per_patient_with_labels.csv',
 'rename_script.sh',
 'segmentation_df.csv',
 'tcga_subtype_data.csv',
 'segmentation_fla_df.csv',
 'segmentation_t1c_df.csv']

# 3 Write out the pngs to ensure that they look ok: 

In [54]:
## Let's investigate just a few of them to ensure that they are correct: 
import nibabel as nib
import matplotlib.pyplot as plt

In [55]:
%matplotlib inline

In [56]:
plotting_dict = {'display_mode':'z', 
                 'cut_coords':[-50,-30,-10, 10, 30, 50], 
                 'cmap':cm.get_cmap('viridis')}

In [ ]:
for idx, row in seg_df.iterrows(): 
    
    original_file     = Path(PATH_TCIA, 'datasets', row['cohort'], row['patient_id'], row['patient_exam'], row['fla'])
    segmentation_file = Path(PATH_TCIA, 'nvidia', 'eval_fla', row['seg_loc'], row['seg_file']) 
    orig_fla_nii      = nib.load(original_file)
    seg_fla_nii       = nib.load(segmentation_file)
    
    png_path = Path(base_dest_path, 'png_output')
    os.makedirs(png_path, exist_ok = True)
    print(f'Saving images for '+str(row['patient_id']))
    plotting.plot_img(orig_fla_nii, title='fla', output_file=str(Path(png_path, str(row['patient_id'])+'flaa.png')), **plotting_dict)
    plotting.plot_stat_map(seg_fla_nii, orig_fla_nii, output_file=str(Path(png_path, str(row['patient_id'])+'seg.png')), title='Segmentation', **plotting_dict)
    

Saving images for TCGA-06-0240
Saving images for TCGA-06-0143
Saving images for TCGA-06-0149
Saving images for TCGA-19-0955
Saving images for TCGA-06-0648
Saving images for TCGA-12-0829
Saving images for TCGA-06-0171
Saving images for TCGA-14-1401
Saving images for TCGA-02-0033
Saving images for TCGA-06-0127
Saving images for TCGA-19-1390
Saving images for TCGA-08-0356
Saving images for TCGA-02-0106
Saving images for TCGA-06-0878
Saving images for TCGA-06-0168
Saving images for TCGA-06-0162
Saving images for TCGA-76-6193
Saving images for TCGA-06-0165
Saving images for TCGA-14-0813
Saving images for TCGA-12-1098
Saving images for TCGA-02-0006
Saving images for TCGA-08-0520
Saving images for TCGA-02-0034
Saving images for TCGA-14-1821
Saving images for TCGA-06-0176
Saving images for TCGA-02-0068
Saving images for TCGA-02-0086
Saving images for TCGA-06-5413
Saving images for TCGA-06-0133
Saving images for TCGA-06-0139
Saving images for TCGA-02-0027
Saving images for TCGA-06-1802
Saving i

/home/sf673542/.local/lib/python3.7/site-packages/nilearn/plotting/displays.py:780: UserWarning: empty mask
  get_mask_bounds(new_img_like(img, not_mask, affine))


Saving images for TCGA-06-0175
Saving images for TCGA-02-0059
Saving images for TCGA-06-1801
Saving images for TCGA-06-2570
Saving images for TCGA-76-4932
Saving images for TCGA-19-0963
Saving images for TCGA-06-0147
Saving images for TCGA-06-0130
Saving images for TCGA-14-1037
